In [28]:
import numpy as np
from selenium import webdriver
from time import sleep
import pandas as pd
import threading
import os
from queue import Queue
import re

In [18]:
os.environ['PATH'] += r"/usr/local/bin/"

In [19]:
base_url = "https://batdongsan.vn/ban-nha/"

In [21]:
def openMultiBrowser(n):
    drivers = []
    for i in range(n):
        driver = webdriver.Firefox()
        drivers.append(driver)
    return drivers

def loadMultiPages(driver,n):
    driver.maximize_window()
    driver.get(f"{base_url}p{n}")
    sleep(5)
   
num_pages = [400,405,410]
def loadMultiBrowsers(drivers_rx): 
    for i,driver in enumerate(drivers_rx):
        t = threading.Thread(target=loadMultiPages,args=(driver,num_pages[i]))
        t.start()

In [22]:
def get_params(text):
    reg_template = {"Area":'Diện tích:\s*(\d+)',
                "Room":'Phòng ngủ:\s*(\d+)',
                "WC":'Phòng WC:\s*(\d+)',
                "Location":'Địa chỉ:\s*(.+)'}
    list_params = []
    for name in reg_template:
        x = re.search(r'{}'.format(reg_template[name]),text)
        if x:
            list_params.append(x.group(1))
        else:
            list_params.append(np.nan)
    return list_params


def get_data(driver):
    report = pd.DataFrame(columns=['Title', 'Price', 'Links',\
            'Diện tích (m2)', 'Số phòng ngủ','Số phòng WC', 'Địa chỉ'\
            ,'Description'])
    n_iter = 2
    for i in range(n_iter):
        driver.implicitly_wait(10)
        try:
            next = driver.find_element('css selector','.uk-pagination [rel="next"]').get_attribute('href')
            elements = driver.find_elements('css selector','.uk-width-medium-7-10 .name [href]')
            elements_price = driver.find_elements('css selector','.price')
        except:
            continue
            
        title = [element.text for element in elements]

        links = [element.get_attribute('href') for element in elements]

        price = [element.text for element in elements_price]
            
        df1 = pd.DataFrame(zip(title,price,links),columns=['Title','Price','Links'])
        
        data = []
        
        n = len(links)

        for j in range(n):
            try:
                driver.get(links[j])
                sleep(5)
                params = driver.find_element('css selector',".param")
                content = driver.find_element('css selector',".content")
            except:
                # df1.drop(j,axis=0,inplace=True)
                continue
            params_ = get_params(params.text)
            params_.append(content.text)
            data.append(params_)

        tmp = np.vstack([item for item in data])
        df2 = pd.DataFrame(tmp,columns=['Diện tích (m2)','Số phòng ngủ','Số phòng WC','Địa chỉ', 'Description'])
        df1 = pd.concat([df1,df2],axis=1)
        report = pd.concat([report,df1],axis=0)
        report.set_index(np.arange(len(report)),inplace=True)
        driver.get(next)
    return report


In [23]:
def runInParallel(func,drivers_rx):
    que = Queue()
    for driver in drivers_rx:
        print("------Running parallel------")
        t1 = threading.Thread(target= lambda q,arg1:q.put(func(arg1)),args=(que,driver))
        t1.start()
    ans = []
    for i in range(len(drivers_rx)):
        try:
            tmp = que.get()
            ans.append(tmp)
        except:
            continue
    return ans

In [24]:
driver_r1 = openMultiBrowser(3)
loadMultiBrowsers(driver_r1)
sleep(5)

In [25]:
list_report = runInParallel(get_data,driver_r1)

------Running parallel------
------Running parallel------
------Running parallel------
